In [ ]:
import os
import json
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from mnist_skeptic_v9 import skeptic_v9
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class CompositeDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        image = torch.tensor(item['composite'], dtype=torch.float32).view(1, 16, 16)
        label = torch.tensor(int(item['true_digit']), dtype=torch.long)
        return image, label

def load_data(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        train_data = []
        test_data = []
        for file in zip_ref.namelist():
            if file.startswith('training_data/training_set/'):
                with zip_ref.open(file) as f:
                    train_data.extend(json.load(f))
            elif file.startswith('training_data/test_set/'):
                with zip_ref.open(file) as f:
                    test_data.extend(json.load(f))
    return train_data, test_data

class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
    
    def forward(self, x):
        outputs = [model(x) for model in self.models]
        return torch.mean(torch.stack(outputs), dim=0)

def train_epoch(ensemble, dataloader, criterion, optimizer, device):
    ensemble.train()
    total_loss = 0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ensemble(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return total_loss / len(dataloader), 100 * correct / total

def evaluate(ensemble, dataloader, criterion, device, show_matrix=False):
    ensemble.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = ensemble(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    
    if show_matrix:
        cm = confusion_matrix(all_labels, all_predictions)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig('confusion_matrix_ensemble.png')
        plt.close()
    
    return total_loss / len(dataloader), accuracy, np.array(all_labels), np.array(all_predictions)

def analyze_confusion_matrix(cm):
    most_confusable_digits = {}
    most_discriminable_digits = {}
    
    for i in range(cm.shape[0]):
        row_sum = cm[i].sum()
        
        if row_sum > 0:
            confusions = cm[i] / row_sum
            
            # Most confusable digit (highest off-diagonal value)
            most_confusable_index = np.argmax(confusions[np.arange(len(confusions)) != i])
            most_confusable_digits[i] = most_confusable_index
            
            # Most discriminable digit (lowest off-diagonal value)
            most_discriminable_index = np.argmin(confusions[np.arange(len(confusions)) != i])
            most_discriminable_digits[i] = most_discriminable_index
    
    return most_confusable_digits, most_discriminable_digits

def finetune_ensemble(ensemble, train_loader, val_loader, num_epochs=50, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ensemble.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ensemble.parameters(), lr=0.0001)
    
    best_val_acc = 0
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(ensemble, train_loader, criterion, optimizer, device)
        val_loss, val_acc, _, _ = evaluate(ensemble, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            torch.save(ensemble.state_dict(), 'best_finetuned_ensemble.pth')
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve == patience:
            print("Early stopping")
            break
    
    # Final evaluation
    ensemble.load_state_dict(torch.load('best_finetuned_ensemble.pth'))
    _, final_accuracy, all_labels, all_predictions = evaluate(ensemble, val_loader, criterion, device, show_matrix=True)
    
    cm = confusion_matrix(all_labels, all_predictions)
    most_confusable_digits, most_discriminable_digits = analyze_confusion_matrix(cm)
    
    print(f'Final Ensemble Accuracy: {final_accuracy:.2f}%')
    print(f'Most Confusable Digits: {most_confusable_digits}')
    print(f'Most Discriminable Digits: {most_discriminable_digits}')

def main():
    # Load data
    train_data, test_data = load_data('training_data.zip')
    
    # Create datasets and dataloaders
    train_dataset = CompositeDataset(train_data)
    test_dataset = CompositeDataset(test_data)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Load ensemble model
    models = []
    for i in range(ord('a'), ord('u')):
        model_name = f'skeptic_v10_{chr(i)}'
        model = skeptic_v9()
        model.load_state_dict(torch.load(f'saved_models/skeptic_v10/{model_name}.pth'))
        models.append(model)
    
    ensemble = EnsembleModel(models)
    
    # Finetune the ensemble
    finetune_ensemble(ensemble, train_loader, test_loader, num_epochs=20)
    
    print("Finetuning completed. Best model saved as 'best_finetuned_ensemble.pth'")

if __name__ == "__main__":
    main()